#Homework

Исходные данные - Yandex Cup 2022 RecSys:

Описание соревнования - https://contest.yandex.ru/yacup/contest/41618/problems/
Данные - https://disk.yandex.ru/d/SI1aAooPn9i8TA
Описание данных - в архиве likes_data.zip три файла:
train - обучающий набор данных. Каждая строка - последовательность id треков, которые лайкнул один пользователь. Гарантируется, что лайки даны в той последовательности, в которой их ставил пользователь.
test - набор тестовых данных. Имеет точно такой же формат, но в каждой строке не хватает последнего лайка, который надо предсказать.
track_artists.csv - информация о исполнителях треков. Гарантируется, что у каждого трека есть ровно один исполнитель. Для треков, у которых фактически несколько исполнителей, мы оставили того, который считается основным исполнителем трека.
Описание сабмита - в качестве решения необходимо отправить файл, в котором для каждого пользователя из test в отдельной строке будет не более 100 треков, разделенных пробелом. Гарантируется, что у каждого пользователя будет только 1 лайк в тесте
Метрика - MRR@100
Промежуточная задача - преобразовать данные в pandas.DataFrame вида {user, item, order}, где order - порядковый номер с конца (0 - самый "свежий" лайк, чем больше order, тем позже был поставлен лайк)

Итоговая задача - построить схему валидации для данного соревнования с учетом особенностей сорвенования

Между train и test не должно быть общих пользователей

Количество фолдов задается через параметр класса n_folds

В test должно быть не более p последних треков (параметр класса p)

In [1]:
!pip install rectools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 89 kB 2.8 MB/s 
     |████████████████████████████████| 13.5 MB 8.4 MB/s 
     |████████████████████████████████| 60 kB 2.1 MB/s 
     |████████████████████████████████| 310 kB 47.1 MB/s 
     |████████████████████████████████| 1.1 MB 48.9 MB/s 
     |████████████████████████████████| 88 kB 8.4 MB/s 
     |████████████████████████████████| 188 kB 40.6 MB/s 
  Created wheel for implicit: filename=implicit-0.4.4-cp37-cp37m-linux_x86_64.whl size=3411065 sha256=7c3ea76f54317786b24a3d08265656a5fad7ddb51698293b3705ec94e3ba7569
  Stored in directory: /root/.cache/pip/wheels/44/7e/7d/a17324ea207cfbe76aca878b5b8ca0aa932cf55d163329be37
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=705376 sha256=be0a62798989c84833f3738a532779ff13afe48fb884ea003b825339c4ea3460
  Stored in directory: /root/.cache/pip/wheels/f8/56/28/5772a3bd

In [2]:
!wget -O yandex.zip 'https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/vghCCsBqC-xQgQ'

--2022-11-24 19:45:22--  https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/vghCCsBqC-xQgQ
Resolving getfile.dokpub.com (getfile.dokpub.com)... 142.132.255.217
Connecting to getfile.dokpub.com (getfile.dokpub.com)|142.132.255.217|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://downloader.disk.yandex.ru/disk/29f05376612ed6a1e418608d4976d8ddd2c3eb81cf0e11dff6e62cdc531e5c59/63800214/hhykUIrqVden_lL2vNRNN9DyK4tLLujpsov9I0NrWo0vDNalChp5V9DMaAvRekyITAGMBTFsXxCEJxKNtXLfMA%3D%3D?uid=0&filename=likes_data.zip&disposition=attachment&hash=v53oKGpCiygFJ679NGHlZOtf5V61F5CVDFb7pIGqnV6UtNWjzyyb6SKMA/64QQ8g/CH%2B%2BsnE5duAiqM/EjDILQ%3D%3D%3A&limit=0&content_type=application%2Fzip&owner_uid=1130000061907806&fsize=361784743&hid=ba2a68ab89688000683587afc4619a0b&media_type=compressed&tknv=v2 [following]
--2022-11-24 19:45:24--  https://downloader.disk.yandex.ru/disk/29f05376612ed6a1e418608d4976d8ddd2c3eb81cf0e11dff6e62cdc531e5c59/63800214/hhykUIrqVden_lL2

In [3]:
!unzip yandex.zip

Archive:  yandex.zip
  inflating: test                    
  inflating: __MACOSX/._test         
  inflating: track_artists.csv       
  inflating: train                   
  inflating: __MACOSX/._train        


In [4]:
import pandas as pd
import numpy as np
from rectools import Columns
from sklearn.model_selection import KFold

In [5]:
train_df = pd.read_csv('train', sep='/n', names=['item_id'])
train_df.head(2)

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,item_id
0,333396 267089 155959 353335 414000 339989 2741...
1,174197 335779 141676 119856 376664 311755 3881...


In [6]:
test_df = pd.read_csv('test', sep='/n', names=['item_id'])
test_df.head(2)

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,item_id
0,454758 382341 240893 280388 362253 436737 2576...
1,294661 374689 182272 102025 70927 86844 280488...


In [7]:
df = pd.concat([train_df, test_df])
len(df) == (len(train_df) + len(test_df))

True

In [8]:
print(df.iloc[0])

item_id    333396 267089 155959 353335 414000 339989 2741...
Name: 0, dtype: object


In [9]:
df['user_id'] = df.index
df.head()

,item_id,user_id
0,333396 267089 155959 353335 414000 339989 2741...,0
1,174197 335779 141676 119856 376664 311755 3881...,1
2,254861 213397 462645 403619 446211 221833 1572...,2
3,132371 199878 357751 92032 94315 435049 260359...,3
4,408028 326575 425054 152422 387296 382570 3505...,4


In [10]:
df = df[:10000] # not enought memory
df = df.assign(item_id=df.item_id.str.split(' ')).explode('item_id').reset_index(drop=True)
print(df)

       item_id  user_id
0       333396        0
1       267089        0
2       155959        0
3       353335        0
4       414000        0
...        ...      ...
813867  324685     9999
813868  298143     9999
813869  218454     9999
813870  153932     9999
813871   43077     9999

[813872 rows x 2 columns]


In [11]:
df = df.reindex(columns=['user_id', 'item_id'])

In [12]:
df.head(60)

,user_id,item_id
0,0,333396
1,0,267089
2,0,155959
3,0,353335
4,0,414000
5,0,339989
6,0,274187
7,0,82655
8,0,128112
9,0,175883


In [13]:
df.rename(
    columns={
        'user_id': Columns.User,
        'item_id': Columns.Item,
    }, 
    inplace=True) 

In [14]:
df[Columns.Rank] = df.groupby(Columns.User).cumcount(ascending=False)
df.head(60)

,user_id,item_id,rank
0,0,333396,53
1,0,267089,52
2,0,155959,51
3,0,353335,50
4,0,414000,49
5,0,339989,48
6,0,274187,47
7,0,82655,46
8,0,128112,45
9,0,175883,44


Между train и test не должно быть общих пользователей
Количество фолдов задается через параметр класса n_folds
В test должно быть не более p последних треков (параметр класса p)

In [18]:
class UsersKFoldPOut():
    def __init__(self, n_folds=3, user_column=Columns.User, p=2, random_seed=23):
        self.n_folds = n_folds
        self.user_column = user_column
        self.p = p
        self.random_seed = random_seed
    
    def split(self, df):
        users = df[self.user_column].unique()
        users_count = len(users)
        
        np.random.seed(self.random_seed)
        np.random.shuffle(users)
        like_rank = df.groupby(Columns.User).cumcount(ascending=False)
        
        fold_sizes = np.full(self.n_folds, users_count // self.n_folds, dtype=int)
        fold_sizes[: users_count % self.n_folds] += 1
        current = 0
        for fold_size in fold_sizes:
            start, stop = current, current + fold_size
            test_fold_users = users[start:stop]
            test_mask = df[self.user_column].isin(test_fold_users) & (like_rank < self.p)
            train_mask = df[self.user_column].isin(~test_fold_users)
            
            yield train_mask, test_mask

In [19]:
cv = UsersKFoldPOut(n_folds=3, p=3)

for i, (train_mask, test_mask) in enumerate(cv.split(df)):
    train = df[train_mask]
    test = df[test_mask]
    print(f'Fold#{i} | Train: {train.shape[0]}, Test: {test.shape[0]}')

Fold#0 | Train: 0, Test: 10002
Fold#1 | Train: 0, Test: 9999
Fold#2 | Train: 0, Test: 9999
